# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [144]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [145]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home
['/workspace/home/event_data/2018-11-10-events.csv', '/workspace/home/event_data/2018-11-20-events.csv', '/workspace/home/event_data/2018-11-09-events.csv', '/workspace/home/event_data/2018-11-11-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-27-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-24-events.csv', '/workspace/home/event_data/2018-11-23-events.csv', '/workspace/home/event_data/2018-11-04-events.csv', '/workspace/home/event_data/2018-11-30-events.csv', '/workspace/home/event_data/2018-11-29-events.csv', '/workspace/home/event_data/2018-11-01-events.csv', '/workspace/home/event_data/2018-11-16-events.csv', '/workspace/home/event_data/2018-11-02-events.csv', '/workspace/home/event_data/2018-11-13-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [146]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [147]:
#Sanity check
print(full_data_rows_list[0])

['Hoobastank', 'Logged In', 'Cierra', 'F', '0', 'Finley', '241.3971', 'free', 'Richmond, VA', 'PUT', 'NextSong', '1.54101E+12', '132', 'Say The Same', '200', '1.54181E+12', '96']


In [148]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [149]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [150]:
# TO-DO: Create a Keyspace 
session.execute("CREATE KEYSPACE IF NOT EXISTS music_ks WITH replication = {'class':'SimpleStrategy', 'replication_factor':1}")

#### Set Keyspace

In [151]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('music_ks')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [152]:
# Before creating the tables, I had a look at the queries and based on that I assumed we need to create 3 table
# 1 table for each query and names them mytable1, mytable2, mytable3 for query 1, 2 & 3 respectively
#-----------------
# Create table N°1
#Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
try:
    session.execute("DROP TABLE IF EXISTS music_ks.mytable1")
except Exception as e:
    print(e)
    
try:
    session.execute("CREATE TABLE IF NOT EXISTS mytable1 (artist varchar, \
                                                     first_name varchar, \
                                                     gender varchar, \
                                                     item_num_sess int, \
                                                     last_name varchar, \
                                                     length float, \
                                                     level varchar, \
                                                     location varchar, \
                                                     session_id int, \
                                                     song varchar, \
                                                     user_id int, \
                                                     PRIMARY KEY (session_id, item_num_sess));")

except Exception as e:
    print(e)
#-----------------
# Create table N°2
#Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
try:
    session.execute("DROP TABLE IF EXISTS music_ks.mytable2")
except Exception as e:
    print(e)
    
try:
    session.execute("CREATE TABLE IF NOT EXISTS mytable2 (artist varchar, \
                                                     first_name varchar, \
                                                     gender varchar, \
                                                     item_num_sess int, \
                                                     last_name varchar, \
                                                     length float, \
                                                     level varchar, \
                                                     location varchar, \
                                                     session_id int, \
                                                     song varchar, \
                                                     user_id int, \
                                                     PRIMARY KEY (session_id, user_id, item_num_sess));")
except Exception as e:
    print(e)
#-----------------
# Create table N°3
#Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
try:
    session.execute("DROP TABLE IF EXISTS music_ks.mytable3")
except Exception as e:
    print(e)
    
try:
    session.execute("CREATE TABLE IF NOT EXISTS mytable3 (artist varchar, \
                                                     first_name varchar, \
                                                     gender varchar, \
                                                     item_num_sess int, \
                                                     last_name varchar, \
                                                     length float, \
                                                     level varchar, \
                                                     location varchar, \
                                                     session_id int, \
                                                     song varchar, \
                                                     user_id int, \
                                                     PRIMARY KEY (song));")
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [153]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    
        ## TO-DO: Assign the INSERT statements into the `query` variable
        #Populating mytable1
        query1 = "INSERT INTO mytable1 (artist, first_name, gender, item_num_sess, last_name, length, level, location, session_id, song, user_id)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
        
        #Populating mytable2
        query2 = "INSERT INTO mytable2 (artist, first_name, gender, item_num_sess, last_name, length, level, location, session_id, song, user_id)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
        
        #Populating mytable3
        query3 = "INSERT INTO mytable3 (artist, first_name, gender, item_num_sess, last_name, length, level, location, session_id, song, user_id)"
        query3 = query3 + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query3, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))



#### Do a SELECT to verify that the data have been inserted into each table

In [154]:
## TO-DO: Add in the SELECT statement to verify the data was entered into table1

query = "SELECT * FROM mytable1 WHERE session_id = 9"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    data = [row for row in rows]
    df = pd.DataFrame(data)
    print(df)
    #print (row.artist, row.first_name, row.gender, row.item_num_sess, row.last_name, row.length, row.level, row.location, row.session_id, row.song, row.user_id)

   session_id  item_num_sess           artist first_name gender last_name  \
0           9              0  Black Eyed Peas     Sylvie      F      Cruz   

       length level                                      location     song  \
0  214.935059  free  Washington-Arlington-Alexandria, DC-VA-MD-WV  Pump It   

   user_id  
0       10  


In [155]:
## TO-DO: Add in the SELECT statement to verify the data was entered into table2

query = "SELECT * FROM mytable2 WHERE session_id = 3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    data = [row for row in rows]
    df = pd.DataFrame(data)
    print(df)
    #print (row.artist, row.first_name, row.gender, row.item_num_sess, row.last_name, row.length, row.level, row.location, row.session_id, row.song, row.user_id)

   session_id  user_id  item_num_sess       artist first_name gender  \
0           3        4              1  The Killers     Alivia      F   

  last_name      length level                location          song  
0   Terrell  246.804443  free  Parkersburg-Vienna, WV  Read My Mind  


In [156]:
## TO-DO: Add in the SELECT statement to verify the data was entered into table3

query = "SELECT * FROM mytable3 WHERE song = 'Read My Mind'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    data = [row for row in rows]
    df = pd.DataFrame(data)
    print(df)
    #print (row.artist, row.first_name, row.gender, row.item_num_sess, row.last_name, row.length, row.level, row.location, row.session_id, row.song, row.user_id)

           song       artist first_name gender  item_num_sess last_name  \
0  Read My Mind  The Killers     Alivia      F              1   Terrell   

       length level                location  session_id  user_id  
0  246.804443  free  Parkersburg-Vienna, WV           3        4  


In [157]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "SELECT artist, song, length, session_id, item_num_sess FROM mytable1 WHERE session_id = 338 AND item_num_sess = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    data = [row for row in rows]
    df = pd.DataFrame(data)
    print(df)
    #print (row.artist, row.song, row.length, row.session_id, row.item_num_sess)

      artist                             song      length  session_id  \
0  Faithless  Music Matters (Mark Knight Dub)  495.307312         338   

   item_num_sess  
0              4  


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [158]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "SELECT artist, song, first_name, last_name, item_num_sess FROM mytable2 WHERE session_id = 182 AND user_id = 10;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    data = [row for row in rows]
    df = pd.DataFrame(data)
    print(df)
    #print (row.item_num_sess, row.artist, row.song, row.first_name, row.last_name)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  item_num_sess  
0     Sylvie      Cruz              0  
1     Sylvie      Cruz              1  
2     Sylvie      Cruz              2  
3     Sylvie      Cruz              3  


In [159]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT first_name, last_name FROM mytable3 WHERE song = 'All Hands Against His Own';"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    data = [row for row in rows]
    df = pd.DataFrame(data)
    print(df)
    #print (row.first_name, row.last_name)
                    

   first_name last_name
0  Jacqueline     Lynch


### Drop the tables before closing out the sessions

In [160]:
## TO-DO: Drop the table before closing out the sessions

In [161]:
try:
    session.execute("DROP TABLE IF EXISTS music_ks.mytable1")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS music_ks.mytable2")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS music_ks.mytable3")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [162]:
session.shutdown()
cluster.shutdown()